In [2]:
import torch
from transformers import AutoTokenizer, AutoModel

/workspace/.pyenv_mirror/user/current/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-04 04:07:01.133425: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-04 04:07:03.634244: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-04 04:07:03.634368: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_

In [3]:
# Load a pre-trained BERT model and tokenizer
model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# Prepare the dataset
dataset = [{
    "context": "Sydney met Carson's mother for the first time last week. He liked her.",
    "questions": [
        "Who did Sydney meet last week?",
        "Did Sydney like Carson's mother?",
        "What happened when Sydney met Carson's mother for the first time?"
    ],
    "answers": [
        "Sydney met Carson's mother.",
        "Sydney liked Carson's mother.",
        "Sydney met Carson's mother for the first time and liked her."
    ]
},
{
    "context": "Kendall had a party at Jordan's house but was found out to not have asked and just broke in.",
    "questions": [
        "Did Kendall break in to Jordan's house?",
        "What happened at Kendall's party at Jordan's house?",
        "Did Kendall have permission to have the party at Jordan's house?"
    ],
    "answers": [
        "Yes, Kendall broke in to Jordan's house.",
        "Kendall had a party at Jordan's house but was found out to not have asked and just broke in.",
        "No, Kendall did not have permission to have the party at Jordan's house."
    ]
}]

In [7]:
# Fine-tune the model
model.train()
for example in dataset:
    context = example["context"]
    questions = example["questions"]
    answers = example["answers"]
    for question, answer in zip(questions, answers):
        input_ids = tokenizer.encode(context + question, add_special_tokens=True)
        input_ids = torch.tensor(input_ids).unsqueeze(0)  # add batch dimension
        answer_start = len(tokenizer.encode(context))
        answer_end = len(tokenizer.encode(answer)) + answer_start
        answer_start = torch.tensor(answer_start).unsqueeze(0)
        answer_end = torch.tensor(answer_end).unsqueeze(0)
        outputs = model(input_ids)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.no_grad()

RuntimeError: grad can be implicitly created only for scalar outputs